In [28]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer, TextIteratorStreamer, TextStreamer

In [ ]:
model_path = "google/bigbird-pegasus-large-arxiv"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = BigBirdPegasusForConditionalGeneration.from_pretrained(model_path)

In [30]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from elastic import elastic_session, Document

In [15]:
session = elastic_session("arxiv-index", "../credentials.json", "../http_ca.crt")
doc = Document(session, 0, filter_path="_source.article").get()

In [47]:
from threading import Thread
from rich.live import Live
from rich.console import Console
from rich.panel import Panel

inputs = tokenizer(doc, return_tensors='pt', truncation=True, max_length=4096)
streamer = TextIteratorStreamer(tokenizer = tokenizer, decode_kwargs={'skip_special_tokens': True})
#prediction = tokenizer.batch_decode(prediction)

#Create new thread that will generate the arguments
t = Thread(target=model.generate, kwargs={**inputs, 'streamer': streamer, 'num_beams': 1})
t.start()

#Render to the console
#===================================================================================================

console = Console()

partial_text = ""
panel = Panel(partial_text, title="Summary", border_style="cyan")

with Live(panel, console=Console()) as live:
    for new_text in streamer:

        if new_text == ".<n>":
            new_text = "\n"

        partial_text += new_text
        panel.renderable = partial_text
        live.update(panel)

Output()

/home/zoukos/ceid/final/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:696: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.8` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
